# Marketing Campaign Results

source: https://platform.stratascratch.com/data-projects/marketing-campaign-results

In [1]:
import pandas as pd

### Readin data

In [ ]:
client = pd.read_csv("./datasets/client_data.csv")
deposit = pd.read_csv("./datasets/deposit_data.csv")
calendar = pd.read_csv("./datasets/calendar_data.csv")

In [6]:
calendar

,gregorian_date,month_name
0,2019-06-01,Month 1
1,2019-06-02,Month 1
2,2019-06-03,Month 1
3,2019-06-04,Month 1
4,2019-06-05,Month 1
...,...,...
148,2019-10-27,Month 5
149,2019-10-28,Month 5
150,2019-10-29,Month 5
151,2019-10-30,Month 5
